In [1]:
import pandas as pd
import Levenshtein

In [2]:
from collections import Counter

In [3]:
!pip install transliterate

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install cyrtranslit

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import transliterate
import cyrtranslit

In [6]:
!cat ./train.csv | shuf | head -n 50000 > sample.csv

shuf: write error: Broken pipe
shuf: write error


In [7]:
!head -n 3 train.csv

id,fullname,country,target,fullname_true
0,AKHMEDOV YGURIY,РОССИЯ,1,AKHMEDOV YURIY
1,ФОЗИЛОВ РАМИЛЬ ГУЛЛОВИЧ,РОССИЯ,1,ФОЗИЛОВ РАМИЛЬ ГУЛОВИЧ


In [8]:
df_sample = pd.read_csv('./sample.csv', header=None, names=['id','fullname','country','target','fullname_true'])

In [9]:
freqs = dict(Counter("".join(df_sample.fullname_true.fillna("").tolist())))

In [10]:
total = sum(freqs.values())

In [11]:
for char in freqs:
    freqs[char] /= total

In [12]:
df_corrections = df_sample[df_sample.target == 1].copy()

In [13]:
#df_corrections.country.value_counts()

In [14]:
df_corrections['distance'] = df_corrections.apply(lambda x: Levenshtein.distance(x.fullname, x.fullname_true), axis=1)

In [15]:
df_corrections.distance.value_counts()

1    22139
2     3109
3      197
4        5
5        1
Name: distance, dtype: int64

In [16]:
Levenshtein.editops('MANAYSANEDGAR', 'MANASYAN EDGAR')

[('replace', 4, 4), ('replace', 5, 5), ('insert', 8, 8)]

In [17]:
df_single_correction = df_corrections[df_corrections.distance == 1].copy()

In [18]:
df_single_correction['edit_op'] = df_single_correction.apply(lambda x: Levenshtein.editops(x.fullname, x.fullname_true)[0], axis=1)

In [19]:
df_replace = df_single_correction[df_single_correction.edit_op.apply(lambda x: x[0]) == 'replace'].copy()

In [20]:
df_replace["from_char"] = df_replace.apply(lambda x: x.fullname[x.edit_op[1]], axis=1)
df_replace["to_char"] = df_replace.apply(lambda x: x.fullname_true[x.edit_op[2]], axis=1)

In [21]:
df_replace['replace'] = df_replace.from_char + "->" + df_replace.to_char

In [22]:
transliterate.translit('ISPARYAN NORIM HURENOVICH', 'ru', reversed=False)

'ИСПАРЫАН НОРИМ ХУРЕНОВИЦХ'

In [23]:
transliterate.translit('ISPARYAN NORIK HURENOVICH', 'ru', reversed=False)

'ИСПАРЫАН НОРИК ХУРЕНОВИЦХ'

In [36]:
for row in df_corrections.itertuples():
    edit_opts = Levenshtein.editops(row.fullname, row.fullname_true)
    num_insert = 0
    for edit_op, from_id, to_id in edit_opts:
        if edit_op == 'insert':
            num_insert += 1
    if num_insert >= 2:
        insert_dist = abs(edit_opts[0][1] - edit_opts[1][1])
        if insert_dist < 1:
            print(row, edit_opts)
        
        

Pandas(Index=1904, id=653369, fullname='ЭРГЕШБАЕВНУРАЙЫМ ТОКТОНАЗАРОВНА', country='КЫРГЫЗСТАН', target=1, fullname_true='ЭРГЕШБАЕВА НУРАЙЫМ ТОКТОНАЗАРОВНА', distance=2) [('insert', 9, 9), ('insert', 9, 10)]
Pandas(Index=7751, id=1857831, fullname='ГОДОВАНЮК А', country='РОССИЯ', target=1, fullname_true='ГОДОВАНЮК АНА', distance=2) [('insert', 11, 11), ('insert', 11, 12)]
Pandas(Index=26525, id=1493504, fullname='МАМАТОВАВИЛЬ ОРОЗБЕКОВИЧ', country='РОССИЯ', target=1, fullname_true='МАМАТОВ РАВИЛЬ ОРОЗБЕКОВИЧ', distance=2) [('insert', 7, 7), ('insert', 7, 8)]
Pandas(Index=35631, id=379643, fullname='ДУЙШАЕАШУРАЛИ КУШАНОВИЧ', country='РОССИЯ', target=1, fullname_true='ДУЙШАЕВ АШУРАЛИ КУШАНОВИЧ', distance=2) [('insert', 6, 6), ('insert', 6, 7)]
Pandas(Index=38957, id=937787, fullname='ТАЛЕКСЕЙ ИННОКЕНТЬЕВИЧ', country='РОССИЯ', target=1, fullname_true='ТЁ АЛЕКСЕЙ ИННОКЕНТЬЕВИЧ', distance=2) [('insert', 1, 1), ('insert', 1, 2)]
Pandas(Index=49086, id=268695, fullname='РУЗИЕВАЙДАР АЛИМБОЕВИЧ'

In [173]:
fullname = "АНДРA ВАСНЕЦОВ"
fullname_true = "АНДРЕЙ ВАСНЕЦОВ"

In [174]:
sorted(Levenshtein.editops(fullname, fullname_true), key=lambda x: (x[0], -x[1]), reverse=True)

[('replace', 4, 5), ('insert', 4, 4)]

In [175]:
encode(fullname, fullname_true)

['', '', '', '', '', '+Е', '', '', '', '', '', '', '', '', '', '']

In [176]:
restore(fullname, encode(fullname, fullname_true))

'АНДРЕA ВАСНЕЦОВ'

In [170]:
def encode(fullname, fullname_true):
    fullname = '#' + fullname + "#"
    fullname_true = '#' + fullname_true + "#"
    target = [''] * len(fullname)
    edit_opts = Levenshtein.editops(fullname, fullname_true)
    edit_opts = sorted(edit_opts, key=lambda x: (x[0], -x[1]), reverse=True)
    for op, src, dst in edit_opts:
        if op == 'delete':
            target[src] = '--'
        if op == 'replace':
            target[src] = fullname_true[dst]
        if op == 'insert':
            target[src] = '+' + fullname_true[dst]
    return target

In [171]:
def restore(fullname, target):
    fullname = '#' + fullname + "#"
    res = []
    for src, tg in zip(fullname, target):
        if tg == '':
            res.append(src)
        elif tg == '--':
            pass
        elif len(tg) ==2 and tg[0] == '+':
            res.append(tg[1])
            res.append(src)
        else:
            res.append(tg)
    res = ''.join(res)
    return res.strip('#')

In [172]:
true_enc = 0
failed_enc = 0
for row in df_corrections.itertuples():
    if row.fullname_true == restore(row.fullname, encode(row.fullname, row.fullname_true)):
        true_enc += 1
    else:
        failed_enc += 1
true_enc, failed_enc

(25415, 36)